# Invoke Endpoint

In [2]:
import boto3

In [27]:
import sagemaker, boto3, json
from sagemaker.session import Session
import IPython
import json
from IPython.core.display import HTML
import base64
import time


In [4]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [5]:

sts = boto3.client('sts')
print(sts.get_caller_identity())

{'UserId': 'AROAV7S4W3BO46HO2V6RH:SageMaker', 'Account': '411437553757', 'Arn': 'arn:aws:sts::411437553757:assumed-role/AmazonSageMaker-ExecutionRole-20240613T154918/SageMaker', 'ResponseMetadata': {'RequestId': 'b384caea-c312-4256-bff0-f179a90acb41', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b384caea-c312-4256-bff0-f179a90acb41', 'content-type': 'text/xml', 'content-length': '470', 'date': 'Sat, 29 Jun 2024 00:04:28 GMT'}, 'RetryAttempts': 0}}


## Helper Code

In [6]:
# Get bucket contents as a list
def list_all_s3_objects(bucket_name:str, prefix:str)-> [str]:
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name,Prefix=prefix)

    objlist = []
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                objlist.append( obj["Key"].replace(prefix, ""))

    return objlist

In [7]:
# download file from s3 and store in local folder (filename is relative to the current folder) 
# images is a dictionary {localfilename : s3file_key}
def download_from_s3(images):
    for filename, image_key in images.items():
        print("Downloading {0} - {1} - {2} - {3}".format(s3_bucket,key_prefix,image_key,filename))
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}{image_key}", filename)    


In [41]:
def infer_image_from_endpoint(endpointName,image_filename):

    #init client
    client = boto3.client('sagemaker-runtime')

    # open the file name
    with open(image_filename, "rb") as file:
        img = file.read()
        body_content = base64.b64decode(img)
        # Call the endpoint with the image in memory. The endpoint is invoked and a result is recorded.
        response = client.invoke_endpoint(
            EndpointName=endpointName,
            ContentType="application/x-image",
            Body=img
        )
        result = json.loads(response['Body'].read().decode())
        predicted_label_probs = result['probabilities']
        positive_prob = predicted_label_probs[0]
        negative_prob = predicted_label_probs[1]
        
        if abs(positive_prob-negative_prob) < .1:
            label = 'Not Sure'
            prob = positive_prob
        elif positive_prob > negative_prob:
            label = 'Positive'
            prob = positive_prob
        else:
            label = 'Negative'
            prob = negative_prob
            
        
    
        display(
                HTML(
                    f'<img src={image_filename} alt={image_filename} align="left" style="width: 450px;"/>'
                    f"<figcaption>Predicted Label: <b>{label}</b> (Probability: {prob}) </figcaption>"
                )
            )
        
    return None

In [42]:
############## Run on the entire folder #####################
##############################################################
        

#---------------------------------------------------------------------------------------------

s3_bucket = f"s3-avalanche-guard"
key_prefix = "data/cv/uibk/ResNetClassify/validation/negative/"
# The name of the SageMaker endpoint running in our account.  
# Note this needs to be in the same region as this running code. 
ENDPOINT_NAME = 'AR-jumpstart-example-FT-tensorflow-ic-i-2024-06-28-17-07-15-823'
#---------------------------------------------------------------------------------------------

# get all images for the prefix
all_images_list = list_all_s3_objects(s3_bucket,key_prefix)


# Now for each image download locally and then run inference, delete after displaying
for idx,img in enumerate(all_images_list):
    if idx>10:
        break
    
    # download the image
    avalanche_images = {"temp/img01.jpg": img}
    print("Downloading {0}".format(avalanche_images))
    download_from_s3(avalanche_images)

    # run inference
    for image_filename in avalanche_images.keys():
        with open(image_filename, "rb") as file:
            infer_image_from_endpoint(ENDPOINT_NAME,image_filename)
    
    #sleep for 5 seconds
    time.sleep(5)
    # now delete the dopwnloaded image